In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fair-esm
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 43.7 MB/s eta 0:00:00


In [4]:
import torch
import esm
import pandas as pd

# Load ESM-2 model  esm1v_t33_650M_UR90S_1()  esm2_t6_8M_UR50D
#https://github.com/facebookresearch/esm
model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Read protein sequences from a FASTA file
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        sequence = ''
        for line in file:
            if line.startswith('>'):
                if sequence:
                    sequences.append(sequence)
                    sequence = ''
            else:
                sequence += line.strip()
        if sequence:
            sequences.append(sequence)
    return sequences

path ='/content/drive/MyDrive/Watashara_Projects/TIP/'
sequences = read_fasta(path+"Features_extraction/TR_IND_Pos_Neg.fasta")

# Reduce batch size (e.g., to 2)
batch_size = 2
batches = [sequences[i:i+batch_size] for i in range(0, len(sequences), batch_size)]

# Initialize list to accumulate representations
all_sequence_representations = []

# Process batches
for batch_idx, batch in enumerate(batches):
    data = [(f'protein{i}', seq) for i, seq in enumerate(batch)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

    # Accumulate representations
    all_sequence_representations.extend(sequence_representations)

# Convert tensor representations to numpy arrays
all_sequence_representations_numpy = [tensor.numpy() for tensor in all_sequence_representations]

# Convert numpy arrays to list of lists
all_sequence_representations_list = [list(arr) for arr in all_sequence_representations_numpy]

# Write to CSV
feat = pd.DataFrame(all_sequence_representations_list)
feat.to_csv(path+'features/sm2_TIP_all.csv', index=False, header=False)
